# Step 1. Data / data set

In [2]:
%%capture
!pip install pyhealth matplotlib seaborn 

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils, datasets
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, WeightedRandomSampler
%matplotlib inline
sns.set_style('darkgrid')
from torch.utils.data.distributed import Dataset
from itertools import chain
from typing import Optional, Tuple, Union, List

import numpy as np
import torch

from pyhealth.datasets import SampleDataset

def split_by_visit(
    dataset: SampleDataset,
    ratios: Union[Tuple[float, float, float], List[float]],
    seed: Optional[int] = None,
    replace: Optional[bool] = True,
   
):
    """Splits the dataset by visit (i.e., samples).

    Args:
        dataset: a `SampleBaseDataset` object
        ratios: a list/tuple of ratios for train / val / test
        seed: random seed for shuffling the dataset

    Returns:
        train_dataset, val_dataset, test_dataset: three subsets of the dataset of
            type `torch.utils.data.Subset`.

    Note:
        The original dataset can be accessed by `train_dataset.dataset`,
            `val_dataset.dataset`, and `test_dataset.dataset`.
    """
    
    if seed is not None:
        np.random.seed(seed)
        rng = np.random.default_rng(seed)
    else:
        rng = np.random.default_rng(12345)
    assert sum(ratios) == 1.0, "ratios must sum to 1.0"
    labels = np.array([sample['label'] for sample in dataset])
    n = len(labels)
    maj, min = 1-sum(labels)/n, sum(labels)/n
    ranges = np.arange(n)
    index = []
    maj_indexes = ranges[labels==0]
    min_indexes = ranges[labels==1]
    for i in ranges:
      #use the same random generator
      choice = rng.choice([0, 1], 1, p=(maj, min), replace=replace)
      if choice == 0:
        index.append(rng.choice(maj_indexes, 1, replace=replace))
      else:
        index.append(rng.choice(min_indexes, 1, replace=replace))

    np.random.shuffle(index)
    train_index = index[: int(len(dataset) * ratios[0])]
    val_index = index[
        int(len(dataset) * ratios[0]) : int(len(dataset) * (ratios[0] + ratios[1]))
    ]
    test_index = index[int(len(dataset) * (ratios[0] + ratios[1])) :]
    train_dataset = torch.utils.data.Subset(dataset, train_index)
    val_dataset = torch.utils.data.Subset(dataset, val_index)
    test_dataset = torch.utils.data.Subset(dataset, test_index)
    return train_dataset, val_dataset, test_dataset

In [4]:
from pyhealth.datasets import MIMIC3Dataset
mimic3_base = MIMIC3Dataset(
  root="https://storage.googleapis.com/pyhealth/Synthetic_MIMIC-III/",
   tables=["DIAGNOSES_ICD", "PROCEDURES_ICD", "PRESCRIPTIONS"],
  code_mapping={"ICD9CM": "CCSCM"},
 )

from pyhealth.tasks import mortality_prediction_mimic3_fn
mimic3_sample = mimic3_base.set_task(mortality_prediction_mimic3_fn)
total_samples = len(mimic3_sample)
min_prop = len([sample for sample in mimic3_sample.samples if sample['label']==1])/total_samples
maj_prop = len([sample for sample in mimic3_sample.samples if sample['label']==0])/total_samples

Mapping codes: 100%|██████████| 49993/49993 [00:08<00:00, 5582.67it/s]
Generating samples for mortality_prediction_mimic3_fn: 100%|██████████| 49993/49993 [00:00<00:00, 458214.53it/s]


In [5]:
mimic3_sample.stat()

Statistics of sample dataset:
	- Dataset: MIMIC3Dataset
	- Task: mortality_prediction_mimic3_fn
	- Number of samples: 2194
	- Number of patients: 2116
	- Number of visits: 2194
	- Number of visits per patient: 1.0369
	- conditions:
		- Number of conditions per sample: 9.6746
		- Number of unique conditions: 255
		- Distribution of conditions (Top-10): [('98', 887), ('101', 839), ('106', 708), ('238', 631), ('108', 584), ('55', 558), ('53', 534), ('49', 505), ('259', 410), ('59', 381)]
	- procedures:
		- Number of procedures per sample: 4.0789
		- Number of unique procedures: 815
		- Distribution of procedures (Top-10): [('3893', 458), ('3961', 419), ('9904', 393), ('8856', 250), ('3615', 234), ('9604', 222), ('966', 213), ('8872', 208), ('9671', 181), ('3722', 177)]
	- drugs:
		- Number of drugs per sample: 35.5228
		- Number of unique drugs: 2424
		- Distribution of drugs (Top-10): [('00338001702', 1023), ('00008084199', 903), ('51079025520', 893), ('00338004938', 887), ('00338004904'

"Statistics of sample dataset:\n\t- Dataset: MIMIC3Dataset\n\t- Task: mortality_prediction_mimic3_fn\n\t- Number of samples: 2194\n\t- Number of patients: 2116\n\t- Number of visits: 2194\n\t- Number of visits per patient: 1.0369\n\t- conditions:\n\t\t- Number of conditions per sample: 9.6746\n\t\t- Number of unique conditions: 255\n\t\t- Distribution of conditions (Top-10): [('98', 887), ('101', 839), ('106', 708), ('238', 631), ('108', 584), ('55', 558), ('53', 534), ('49', 505), ('259', 410), ('59', 381)]\n\t- procedures:\n\t\t- Number of procedures per sample: 4.0789\n\t\t- Number of unique procedures: 815\n\t\t- Distribution of procedures (Top-10): [('3893', 458), ('3961', 419), ('9904', 393), ('8856', 250), ('3615', 234), ('9604', 222), ('966', 213), ('8872', 208), ('9671', 181), ('3722', 177)]\n\t- drugs:\n\t\t- Number of drugs per sample: 35.5228\n\t\t- Number of unique drugs: 2424\n\t\t- Distribution of drugs (Top-10): [('00338001702', 1023), ('00008084199', 903), ('5107902552

# Step 1b. Sampling process and Set TASKS

In [6]:
import torch
from torch.utils.data import TensorDataset, DataLoader, WeightedRandomSampler
from tqdm import tqdm
from pyhealth.datasets.splitter import split_by_patient
from pyhealth.datasets import split_by_patient, get_dataloader

# data split
BATCH_SIZE = 128
train_dataset, val_dataset, test_dataset = split_by_patient(mimic3_sample, [0.8, 0.1, 0.1])
train_loader, val_loader, test_loader = get_dataloader(train_dataset, batch_size=BATCH_SIZE),\
                                         get_dataloader(val_dataset, batch_size=BATCH_SIZE),\
                                        get_dataloader(test_dataset, batch_size=BATCH_SIZE),

In [7]:
next(iter(train_dataset))

{'visit_id': '144281',
 'patient_id': '42063',
 'conditions': [['122', '103', '49', '151', '2616']],
 'procedures': [['3723']],
 'drugs': [['00409125830',
   '00338004904',
   '00904516561',
   '63323026201',
   '00002821501',
   '63653117103',
   '51991045757',
   '51079045120',
   '00904504561',
   '00182845389',
   '00003029305']],
 'label': 0}

In [8]:

# def split_by_visit(
#     dataset: SampleDataset,
#     ratios: Union[Tuple[float, float, float], List[float]],
#     seed: Optional[int] = None,
#     replace: Optional[bool] = True,
   
# ):
#     """Splits the dataset by visit (i.e., samples).

#     Args:
#         dataset: a `SampleBaseDataset` object
#         ratios: a list/tuple of ratios for train / val / test
#         seed: random seed for shuffling the dataset

#     Returns:
#         train_dataset, val_dataset, test_dataset: three subsets of the dataset of
#             type `torch.utils.data.Subset`.

#     Note:
#         The original dataset can be accessed by `train_dataset.dataset`,
#             `val_dataset.dataset`, and `test_dataset.dataset`.
#     """
    
#     if seed is not None:
#         np.random.seed(seed)
#         rng = np.random.default_rng(seed)
#     else:
#         rng = np.random.default_rng(12345)
#     assert sum(ratios) == 1.0, "ratios must sum to 1.0"
#     labels = np.array([sample['label'] for sample in dataset])
#     n = len(labels)
#     maj, min = 1-sum(labels)/n, sum(labels)/n
#     ranges = np.arange(n)
#     index = []
#     maj_indexes = ranges[labels==0]
#     min_indexes = ranges[labels==1]
#     for i in ranges:
#       #use the same random generator
#       choice = rng.choice([0, 1], 1, p=(0.5, 0.5), replace=replace)
#       if choice == 0:
#         index.append(rng.choice(maj_indexes, 1, replace=replace).item())
#       else:
#         index.append(rng.choice(min_indexes, 1, replace=replace).item())
#     np.random.shuffle(index)
#     train_index = index[: int(len(dataset) * ratios[0])]
#     val_index = index[
#         int(len(dataset) * ratios[0]) : int(len(dataset) * (ratios[0] + ratios[1]))
#     ]
#     test_index = index[int(len(dataset) * (ratios[0] + ratios[1])) :]
#     train_dataset = torch.utils.data.Subset(dataset, train_index)
#     val_dataset = torch.utils.data.Subset(dataset, val_index)
#     test_dataset = torch.utils.data.Subset(dataset, test_index)
#     return train_dataset, val_dataset, test_dataset
# train_dataset, val_dataset, test_dataset = split_by_visit(mimic3_sample, 
#                                                           ratios=[0.8, 0.1, 0.1])
# train_loader, val_loader, test_loader = get_dataloader(train_dataset, batch_size=BATCH_SIZE),\
#                                         get_dataloader(val_dataset, batch_size=BATCH_SIZE),\
#                                         get_dataloader(test_dataset, batch_size=BATCH_SIZE),

# Step 3. Models / Customized models

In [9]:

class CustomLoss(nn.Module):
  def __init__(self):
    super(CustomLoss, self).__init__()
  def forward(self, output, target):
    target = torch.LongTensor(target)
    criterion = nn.CrossEntropyLoss()
    loss = criterion(output, target)
    return loss 
  def cross_entropy(y_hat, y):
    l = -y * torch.log(y_hat)
    return l.sum(dim=1).mean()
  def criterion(output, target):
    num_class_0, num_class_1 = len(target==0), len(target==1)
    K = F.l1_loss(output, target)
    delta_c_0, delta_c_1 = K/ ((num_class_0)**(1/4)), K/ ((num_class_1)**(1/4))
    delta_c = torch.tensor(delta_c_0, delta_c_1)
    ce_with_density = cross_entropy(output + delta_c, target)
    return ce_with_density.sum(dim=1).mean()



In [20]:
# device = 'cpu'
# model = MyPyhealthGRU(
#     dataset=mimic3_sample,
#     feature_keys=['conditions', 'drugs', 'procedures'],
#     label_key='label',
#     mode='binary',
#     embedding_dim=128,
#     hidden_dim=64

# )
# model.to(device)

from pyhealth.models import Deepr
from pyhealth.trainer import Trainer
model = Deepr(
         dataset=mimic3_sample,
        feature_keys=['conditions', 'drugs', 'procedures'],
        label_key="label",
        mode="binary",   )
from pyhealth.trainer import Trainer
trainer = Trainer(model=model)
trainer.train(
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    epochs=50,
    monitor = "pr_auc",
)



Deepr(
  (embeddings): ModuleDict(
    (conditions): Embedding(258, 128, padding_idx=0)
    (drugs): Embedding(2427, 128, padding_idx=0)
    (procedures): Embedding(818, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (cnn): ModuleDict(
    (conditions): DeeprLayer(
      (conv): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
    )
    (drugs): DeeprLayer(
      (conv): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
    )
    (procedures): DeeprLayer(
      (conv): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
    )
  )
  (fc): Linear(in_features=384, out_features=1, bias=True)
)
Metrics: None
Device: cuda

Training:
Batch size: 128
Optimizer: <class 'torch.optim.adam.Adam'>
Optimizer params: {'lr': 0.001}
Weight decay: 0.0
Max grad norm: None
Val dataloader: <torch.utils.data.dataloader.DataLoader object at 0x7f1c256ba070>
Monitor: pr_auc
Monitor criterion: max
Epochs: 50



Epoch 0 / 50:   0%|          | 0/14 [00:00<?, ?it/s]

--- Train epoch-0, step-14 ---
loss: 0.2158
Evaluation: 100%|██████████| 2/2 [00:00<00:00, 79.27it/s]
--- Eval epoch-0, step-14 ---
pr_auc: 0.0449
roc_auc: 0.5866
f1: 0.0000
loss: 0.1608
New best pr_auc score (0.0449) at epoch-0, step-14



Epoch 1 / 50:   0%|          | 0/14 [00:00<?, ?it/s]

--- Train epoch-1, step-28 ---
loss: 0.1495
Evaluation: 100%|██████████| 2/2 [00:00<00:00, 80.47it/s]
--- Eval epoch-1, step-28 ---
pr_auc: 0.0462
roc_auc: 0.5767
f1: 0.0000
loss: 0.1523
New best pr_auc score (0.0462) at epoch-1, step-28



...

--- Train epoch-47, step-672 ---
loss: 0.0003
Evaluation: 100%|██████████| 2/2 [00:00<00:00, 77.03it/s]
--- Eval epoch-47, step-672 ---
pr_auc: 0.0519
roc_auc: 0.3818
f1: 0.0000
loss: 0.2836



Epoch 48 / 50:   0%|          | 0/14 [00:00<?, ?it/s]

--- Train epoch-48, step-686 ---
loss: 0.0003
Evaluation: 100%|██████████| 2/2 [00:00<00:00, 76.94it/s]
--- Eval epoch-48, step-686 ---
pr_auc: 0.0590
roc_auc: 0.3818
f1: 0.0000
loss: 0.2850



Epoch 49 / 50:   0%|          | 0/14 [00:00<?, ?it/s]

--- Train epoch-49, step-700 ---
loss: 0.0003
Evaluation: 100%|██████████| 2/2 [00:00<00:00, 77.43it/s]
--- Eval epoch-49, step-700 ---
pr_auc: 0.0590
roc_auc: 0.3812
f1: 0.0000
loss: 0.2863
Loaded best model


In [ ]:
import sys
with open('file', 'w') as sys.stdout:
  print(
    trainer.train(
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    epochs=50,
    monitor = "pr_auc",
))

In [ ]:
!ls